MNIST Data Exploration

In [25]:

import numpy as np
import pandas as pd
import csv
from sklearn.model_selection import train_test_split
import xgboost
from sklearn import metrics

#all the data is under the same folder
import os
print(os.listdir("../mnist_data_exploration"))


['.git', '.gitignore', '.ipynb_checkpoints', 'LICENSE', 'MNIST_EXPLORATION_WITH_XGBOOST.ipynb', 'README.md', 'sample_submission.csv', 'test.csv', 'train.csv']


In [26]:
df_train = pd.read_csv('../mnist_data_exploration/train.csv')
y = df_train.label.values
X = df_train.drop('label', axis=1).values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)
#df_train.head()

In [27]:
#print(X_train)
dtrain = xgboost.DMatrix(X_train, label=y_train)
dval = xgboost.DMatrix(X_val, label=y_val)

params = {
    'max_depth': 5,                 # the maximum depth of each tree
    'eta': 0.5,                     # the training step for each iteration
    'eta_decay': 0.9,
    'min_eta': 0.05,
    'silent': 0,                    # logging mode - quiet
    'objective': 'multi:softmax',   # multiclass classification using the softmax objective
    'num_class': 10,                 # the number of classes that exist in this datset
    'early_stopping_rounds': 10,
    'n_estimators': 5000
}  

def eval_acc(preds, dtrain):
    labels = dtrain.get_label()
    return 'acc', metrics.accuracy_score(labels, preds)

results = {}
model = xgboost.train(params, dtrain, num_boost_round=1, evals=[(dval, 'val')], evals_result=results, feval=eval_acc)

[0]	val-merror:0.18381	val-acc:0.81619


In [31]:
df_test = pd.read_csv('../mnist_data_exploration/test.csv')
#df_test.head()
x_test = df_test.values
dtest = xgboost.DMatrix(x_test)

In [32]:
pred_values = model.predict(dtest)
print(pred_values.shape)
pred_values[:2]

(28000,)


array([2., 0.], dtype=float32)

In [35]:
df_sub = pd.read_csv('../mnist_data_exploration/sample_submission.csv')
df_sub.head()
df_sub.Label = pred_values.astype(np.int32)
df_sub.head()
#df_sub.to_csv('submission.csv', index=False)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,2
